In [1]:
#importing data
import pandas as pd
data = pd.read_csv("/home/roopal/Desktop/witsy/uber_data4.csv")

In [2]:
data.head()

,Unnamed: 0,cons,pros
0,0,- Business stagnation in India and overseas\r\...,Low gender wage gap in entry-level and mid-man...
1,1,There is no structure defined for work. One ha...,Flat hierarchy. Benefits are good. Food in the...
2,2,Good to join as fresher,Nice experience easily accessible and easily l...
3,3,As of now nothing as i haven't noticed anythin...,"Unmatched employee benefits,Good working ambei..."
4,4,1. Too much work for an individual,1. Superb culture \r\n2. Loads of problems to ...


In [3]:
import gensim
from gensim.utils import simple_preprocess
from nltk.stem.porter import *
import numpy as np
import nltk
np.random.seed(200)
import spacy
from nltk.tokenize import word_tokenize

In [4]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [5]:
print(word_tokenize(data['cons'].values[0]))

['-', 'Business', 'stagnation', 'in', 'India', 'and', 'overseas\\r\\n-', 'Bureaucracy']


In [6]:
word = []
for i in range(len(data['cons'])):
    word.append((word_tokenize(data['cons'].values[i])))

In [7]:
def lemmatize(line):
    return [token.lemma_ for token in line 
                      if not token.is_punct ]

In [8]:
# Converting integer list to string list 
# and joining the list using join() 
def convert(list): 
    res = (" ".join(list))
      
    return res 

In [9]:
#lowerd and stemmed by normalization
normal = []
for text in word:
    sample_review_normalized = lemmatize(nlp(convert(text)))
    normal.append(sample_review_normalized)

In [10]:
normal

[['business',
  'stagnation',
  'in',
  'India',
  'and',
  'overseas\\r\\n-',
  'Bureaucracy'],
 ['there',
  'be',
  'no',
  'structure',
  'define',
  'for',
  'work',
  'one',
  'have',
  'to',
  'figure',
  'out',
  'by',
  '-PRON-'],
 ['good', 'to', 'join', 'as', 'fresher'],
 ['as',
  'of',
  'now',
  'nothing',
  'as',
  'i',
  'have',
  "n't",
  'notice',
  'anything',
  'yet'],
 ['1', 'too', 'much', 'work', 'for', 'an', 'individual'],
 ['no', 'negative', 'word', 'about', 'this', 'company'],
 ['net', 'earning', 'reduce', 'to', '12', 'month'],
 ['no',
  'learning',
  'Biased',
  'manager',
  'no',
  'Meritocracy',
  'disgraceful',
  'integrity',
  'of',
  'leadership',
  'all',
  'gyaan',
  'and',
  'no',
  'action',
  'by',
  'management'],
 ['-PRON-',
  'could',
  'have',
  'a',
  'more',
  'comprehensive',
  'training',
  'process',
  'for',
  'lateral',
  'hire'],
 ['-PRON-',
  'can',
  'expect',
  'sudden',
  'work',
  'call',
  'which',
  'can',
  'be',
  'hectic',
  'at',


In [11]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *



def preprocess(text):
    result = [token for token in gensim.utils.simple_preprocess(text) 
              # Remove stop words and words less than 3 characters long
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]
    return result

In [12]:
for i in range(len(word)):
    word[i] = preprocess(convert(word[i]))

In [13]:
def l(text):
    c = [text.remove('pron')]
    return c

In [14]:
for i in range(len(word)):
    if 'pron' in word[i]:
        word[i] = l((word[i]))

In [15]:
for i in range(len(word)):
    if word[i] == [None]:
        del word[i]

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(word, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[word], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[word[15]]])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
make_bigrams(word)

['growth', 'constraint', 'terms', 'promotion']


[['business', 'stagnation', 'india', 'overseas', 'bureaucracy'],
 ['structure', 'defined', 'work', 'figure'],
 ['good', 'join', 'fresher'],
 ['noticed'],
 ['work', 'individual'],
 ['negative', 'words', 'company'],
 ['earning', 'reduces', 'month'],
 ['learning',
  'biased',
  'managers',
  'meritocracy',
  'disgraceful',
  'integrity',
  'leadership',
  'gyaan',
  'action',
  'management'],
 ['comprehensive', 'training', 'process', 'lateral', 'hires'],
 ['expect', 'sudden', 'work', 'calls', 'hectic', 'times'],
 ['work',
  'line',
  'exciting',
  'work',
  'engineering',
  'happens',
  'india',
  'nthe',
  'engineering',
  'leadership',
  'blindsided',
  'cultural',
  'values',
  'facilitate',
  'posters',
  'motivational',
  'slogans',
  'ground',
  'ample',
  'politics',
  'need',
  'deal',
  'start',
  'getting',
  'stuff'],
 ['remote', 'location', 'impact', 'ability', 'contribute'],
 ['party', 'roll'],
 ['work',
  'good',
  'ownership',
  'core',
  'components',
  'lacks',
  'think',

In [17]:
dictionary = gensim.corpora.Dictionary(word)

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in word]

In [19]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=8, 
                                       id2word = dictionary, 
                                       passes = 10, 
                                       workers=2)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.011*"company" + 0.007*"empresa" + 0.007*"time" + 0.007*"uber" + 0.005*"process" + 0.005*"growth" + 0.005*"employees" + 0.005*"working" + 0.004*"structure" + 0.004*"communication"


Topic: 1 Word: 0.021*"company" + 0.011*"like" + 0.009*"uber" + 0.008*"culture" + 0.008*"employees" + 0.008*"management" + 0.006*"time" + 0.006*"times" + 0.005*"fast" + 0.005*"better"


Topic: 2 Word: 0.027*"uber" + 0.012*"time" + 0.010*"work" + 0.009*"need" + 0.007*"people" + 0.007*"high" + 0.006*"working" + 0.006*"things" + 0.006*"leadership" + 0.005*"company"


Topic: 3 Word: 0.021*"company" + 0.017*"work" + 0.014*"management" + 0.013*"people" + 0.010*"team" + 0.009*"cons" + 0.008*"uber" + 0.007*"experience" + 0.007*"great" + 0.007*"hours"


Topic: 4 Word: 0.011*"uber" + 0.007*"empresa" + 0.006*"startup" + 0.006*"culture" + 0.006*"para" + 0.005*"salary" + 0.005*"working" + 0.005*"people" + 0.005*"things" + 0.005*"company"


Topic: 5 Word: 0.060*"work" + 0.023*"life" + 0.021*"balance" + 0.0

In [21]:
 lda_model.get_term_topics("company", minimum_probability=None)

[(0, 0.010880922),
 (1, 0.020996751),
 (3, 0.020783374),
 (5, 0.011500756),
 (6, 0.013665948)]

In [22]:
lda_model. get_topic_terms(2, topn=10)

[(107, 0.026927102),
 (167, 0.012348256),
 (8, 0.009840605),
 (52, 0.009319607),
 (101, 0.0074846265),
 (373, 0.0065819323),
 (213, 0.0063921087),
 (86, 0.006352324),
 (25, 0.0055308663),
 (14, 0.005396213)]

In [23]:
lda_model. show_topic(4, topn=10)

[('uber', 0.011434201),
 ('empresa', 0.006603851),
 ('startup', 0.0060191345),
 ('culture', 0.005831808),
 ('para', 0.005625131),
 ('salary', 0.005418323),
 ('working', 0.0052919313),
 ('people', 0.0052890186),
 ('things', 0.005280522),
 ('company', 0.0051804697)]

In [24]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.144932  0.000261       1        1  25.897179
3      0.066567  0.014629       2        1  17.035406
2      0.013080 -0.104805       3        1  11.607918
7      0.025936  0.119980       4        1  11.181431
1     -0.029001 -0.025100       5        1   9.370506
0     -0.104203  0.016468       6        1   9.199656
6     -0.020352 -0.050161       7        1   8.226624
4     -0.096958  0.028728       8        1   7.481289, topic_info=     Category        Freq         Term       Total  loglift  logprob
114   Default  153.000000         life  153.000000  30.0000  30.0000
110   Default  137.000000      balance  137.000000  29.0000  29.0000
8     Default  545.000000         work  545.000000  28.0000  28.0000
107   Default  264.000000         uber  264.000000  27.0000  27.0000
223   Default  155.000000         long  155.000000  26.0000  26.0000
211   Default  191.000000        hours  191.000000  25.0000  25.0000
197   Default   29.000000       shifts   29.000000  24.0000  24.0000
14    Default  310.000000      company  310.000000  23.0000  23.0000
27    Default  170.000000   management  170.000000  22.0000  22.0000
186   Default  130.000000         fast  130.000000  21.0000  21.0000
459   Default  128.000000         like  128.000000  20.0000  20.0000
4358  Default   48.000000      empresa   48.000000  19.0000  19.0000
275   Default   70.000000         cons   70.000000  18.0000  18.0000
80    Default   79.000000        great   79.000000  17.0000  17.0000
88    Default  101.000000       growth  101.000000  16.0000  16.0000
329   Default   43.000000      startup   43.000000  15.0000  15.0000
77    Default   63.000000        think   63.000000  14.0000  14.0000
156   Default   94.000000      culture   94.000000  13.0000  13.0000
189   Default   32.000000        paced   32.000000  12.0000  12.0000
94    Default   42.000000       salary   42.000000  11.0000  11.0000
39    Default   61.000000        times   61.000000  10.0000  10.0000
1266  Default   24.000000        makes   24.000000   9.0000   9.0000
130   Default  102.000000    employees  102.000000   8.0000   8.0000
731   Default   42.000000   experience   42.000000   7.0000   7.0000
4414  Default   46.000000         para   46.000000   6.0000   6.0000
98    Default   59.000000       little   59.000000   5.0000   5.0000
52    Default  110.000000         need  110.000000   4.0000   4.0000
483   Default   47.000000    processes   47.000000   3.0000   3.0000
4456  Default   25.000000         como   25.000000   2.0000   2.0000
7     Default   38.000000    structure   38.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1160   Topic8    2.842004        curve    5.901588   1.8621  -6.4784
94     Topic8   10.023334       salary   42.829765   1.1404  -5.2180
4414   Topic8   10.405908         para   46.543137   1.0948  -5.1805
655    Topic8    3.580104       global    8.595625   1.7169  -6.2475
12     Topic8    2.120153      noticed    3.753813   2.0215  -6.7714
2247   Topic8    2.118254        solid    3.759883   2.0190  -6.7723
4865   Topic8    4.180977        entre   11.959200   1.5418  -6.0923
26     Topic8    6.863734     learning   29.684889   1.1284  -5.5966
107    Topic8   21.152082         uber  264.777252   0.0656  -4.4711
4458   Topic8    2.021097        hasta    3.726643   1.9809  -6.8192
857    Topic8    2.220600         wear    4.432818   1.9015  -6.7251
156    Topic8   10.788238      culture   94.910576   0.4183  -5.1444
143    Topic8    8.960677        place   68.682487   0.5561  -5.3300
86     Topic8    9.768416       things   96.207245   0.3054  -5.2437
966    Topic8    4.340838        shift   17.209005   1.2154  -6.0548
39     Topic8    7.503728        times   61.386044   0.4910  -5.5075
213    Topic8    9.789522      working  121.676506   0.0727  -5.2416
101    Topic8    9.784134       peop

In [26]:
{0:'Job_Security_and_advancement',1:'Culture and team',2:'Work life balance',3:'Salary & benefits',4:'Business outlook & vision',5:'Harassment | discrimination | bias' , 6:'Technology & Product',7:'management'}

{0: 'Job_Security_and_advancement',
 1: 'Culture and team',
 2: 'Work life balance',
 3: 'Salary & benefits',
 4: 'Business outlook & vision',
 5: 'Harassment | discrimination | bias',
 6: 'Technology & Product',
 7: 'management'}